# EXTRACT

## setting variables

In [1]:

import pandas as pd
import numpy as np
import datetime as dt
import logging
import os
import awswrangler as awr

In [2]:
df_analise = None
df_anterior = None
df_conferencia = None
df_cancelamentos = None
today = pd.Timestamp.today().date()

if today.weekday() == 0:
    yesterday = today - pd.Timedelta(days=3)
else:
    yesterday = today - pd.Timedelta(days=1)

if today.weekday() == 0:
    dbf_yesterday = today - pd.Timedelta(days=4)
else:
    dbf_yesterday = today - pd.Timedelta(days=2)

## extracting

In [3]:
xlsx_anterior = rf"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\bkp_activation\placas_movimentacoes_{yesterday}.xlsx"
xlsx_analise = rf"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\bkp_activation\placas_movimentacoes_{today}.xlsx"

df_anterior = pd.read_excel(xlsx_anterior, engine='openpyxl', sheet_name='ATIVAÇÕES')
df_analise = pd.read_excel(xlsx_analise, engine='openpyxl', sheet_name='ATIVAÇÕES')




In [4]:
set_analise = set(df_analise['chassi'])
set_anterior = set(df_anterior['chassi'])

novos_mask = ~df_analise['chassi'].isin(set_anterior)

desativados_mask = ~df_anterior['chassi'].isin(set_analise)

In [5]:
print(f"Shape de df_anterior: {df_anterior.shape}")
print(f"Shape de df_analise: {df_analise.shape}")
print(f"Quantidade de novos: {novos_mask.sum()}")
print(f"Quantidade de desativados: {desativados_mask.sum()}")


Shape de df_anterior: (33374, 18)
Shape de df_analise: (33381, 18)
Quantidade de novos: 91
Quantidade de desativados: 84


In [6]:

df_analise.loc[novos_mask, 'status_beneficio'] = 'NOVO'
df_analise['status_beneficio'].value_counts()



status_beneficio
ATIVO        33274
NOVO            91
RENOVAÇÃO        9
MIGRAÇÃO         7
Name: count, dtype: int64

interpretação de cancelamentos: como tem 'CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO' entre outros status que são suspensivos, a intepretação de cancelamentos pode ser mais ampla do que apenas CANCELADO, dito isso CANCELAMENTOS AQUI SE ENTENDE COMO TODAS AS PLACAS QUE NÃO ESTÃO ATIVAS MAIS, PARA QUE SEJA REFORÇADA A COMPREENSÃO VOLTADA PARA PLACAS QUE ESTÃO EFETIVAMENTE ATIVAS, OU NÃO

conclusão: o melhor seria denominá-las de desativadas, e para fins de não atrapalhar uma compreensão já consolidada, no resumo final pode-se referir a elas como "canceladas" 

In [7]:
df_desativadas = df_anterior.loc[desativados_mask]
df_desativadas.shape[0]


84

# LOAD

In [10]:
xlsx_analise = rf"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_movimentacoes_{today}.xlsx"
df_analise.to_excel(xlsx_analise, engine='openpyxl', index=False)
path_desativadas = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_desativadas.xlsx"
df_desativadas.to_excel(path_desativadas, engine='openpyxl', index=False)